In [1]:
%matplotlib inline
import glob
import os
import pandas as pd
import json
import numpy as np
import scipy.stats as ss
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
import matplotlib.pyplot as plt
import math
import warnings
warnings.simplefilter('ignore')


# Nash Bargaining Problem



In [2]:
def poi_bargain(a, b):
    prop = a.sum() / (a.sum() + b.sum())
    return np.nan_to_num((prop - 0.5) * 2)
    

## Test Cases

In [3]:
#Case when there are 1 stronger to 1 weaker
p1 = np.asarray([0.8,   0.7,  0.9 ])
p2 = np.asarray([0.2,   0.3,  0.1 ])

poi_bargain(p1, p2)

0.5999999999999999

In [4]:

#Case when there are 2 stronger
p1 = np.asarray([0.8,   0.7,  0.9 ])
p2 = np.asarray([0.6,   0.8,  0.8 ])

poi_bargain(p1, p2)

0.04347826086956519

In [5]:
#Case when there are 2 stronger same rate
p1 = np.asarray([0.8,   0.7,  0.9 ])
p2 = np.asarray([0.8,   0.7,  0.9 ])

poi_bargain(p1, p2)

0.0

In [6]:
#Case when there are 2 weaker
p1 = np.asarray([0.2,   0.3,  0.1 ])
p2 = np.asarray([0.1,   0.2,  0.1 ])

poi_bargain(p1, p2)

0.19999999999999996

In [7]:
#Case when there are 1 weaker to 1 stronger
p1 = np.asarray([0.2,   0.3,  0.1 ])
p2 = np.asarray([0.8,   0.7,  0.9 ])

poi_bargain(p1, p2)

-0.6000000000000001

In [8]:
#Case when there are 2 weaker same rate
p1 = np.asarray([0.2,   0.3,  0.1 ])
p2 = np.asarray([0.2,   0.3,  0.1 ])

poi_bargain(p1, p2)

0.0

In [9]:
#Case when there are average 1 weaker
p1 = np.asarray([0.5,   0.5,  0.5 ])
p2 = np.asarray([0.2,   0.3,  0.1 ])

poi_bargain(p1, p2)

0.4285714285714286

In [10]:
#Case when there are average 1 stronger
p1 = np.asarray([0.5,   0.5,  0.5 ])
p2 = np.asarray([0.8,   0.7,  0.9 ])

poi_bargain(p1, p2)

-0.23076923076923073

In [11]:
#Case when there are 2 in average
p1 = np.asarray([0.5,   0.5,  0.5 ])
p2 = np.asarray([0.5,   0.5,  0.5 ])

poi_bargain(p1, p2)

0.0

In [12]:

#Case when both 0
p1 = np.asarray([0,   0,  0 ])
p2 = np.asarray([0,   0,  0 ])

poi_bargain(p1, p2)

0.0

In [13]:
#Case when both 1
p1 = np.asarray([1,   1,  1 ])
p2 = np.asarray([1,   1,  1 ])

poi_bargain(p1, p2)

0.0

In [14]:
#Case when there upper and lower bound
p1 = np.asarray([1,   1,  1 ])
p2 = np.asarray([0,   0,  0 ])

poi_bargain(p1, p2)

1.0

In [15]:
#Case mixed
p1 = np.asarray([0.9,   0.2,  0.4,  0.6,   0.7,  0.1 ])
p2 = np.asarray([0.6,   0.2,  0.5,  0.4,   0.4,  0.3 ])

poi_bargain(p1, p2)

0.09433962264150941

In [35]:
df = pd.read_csv("graphs/power_for_02-07-17_18H.csv")
df['in_power'] = df['in_power'].apply(pd.to_numeric)
df['out_power'] = df['out_power'].apply(pd.to_numeric)

In [44]:
df.head()

,in,in_amenity,in_power,out,out_amenity,out_power,power
0,5529666646,cafe,0.965344,411290141,bank,3.0,0.243445
1,411290141,bank,3.000000,2624201223,pub,1.0,0.750000
2,411290141,bank,3.000000,5862416790,bicycle_rental,1.0,0.750000
3,2624201223,pub,1.000000,3552405837,bar,1.0,0.500000
4,5862416790,bicycle_rental,1.000000,5276126259,restaurant,1.0,0.500000


In [45]:
def poi_bargain(a, b):
    prop = a / (a + b)
    #return np.nan_to_num((prop - 0.5) * 2)
    return prop
    

In [46]:
df['power'] = df.apply(lambda row: poi_bargain(row['in_power'], row['out_power']), axis=1)

In [47]:
df.sort_values(['power'], ascending=False)

,in,in_amenity,in_power,out,out_amenity,out_power,power
57,412112860,bank,1.000000,2337784209,bicycle_parking,0.000000,1.000000
52,2639373746,restaurant,1.000000,529117814,pub,0.000000,1.000000
45,1243846551,post_box,1.000000,4616049555,clinic,0.000000,1.000000
46,3581990318,car_sharing,1.000000,206723692,theatre,0.000000,1.000000
47,2722757446,cafe,1.000000,4595556508,restaurant,0.000000,1.000000
50,111586252,parking,1.000000,995035559,fast_food,0.000000,1.000000
51,468705543,restaurant,1.000000,1932984609,restaurant,0.000000,1.000000
48,5301492675,cafe,1.000000,2340006882,fast_food,0.000000,1.000000
53,4039511540,restaurant,1.000000,288472567,restaurant,0.000000,1.000000
54,390197717,post_box,1.000000,390197014,bicycle_parking,0.000000,1.000000


In [48]:
df.sort_values(['in_power','power'], ascending=False)

,in,in_amenity,in_power,out,out_amenity,out_power,power
34,2718062345,pharmacy,3.000000,2612990005,restaurant,0.607104,0.831692
1,411290141,bank,3.000000,2624201223,pub,1.000000,0.750000
2,411290141,bank,3.000000,5862416790,bicycle_rental,1.000000,0.750000
11,358805065,place_of_worship,3.000000,5352315966,fast_food,1.000000,0.750000
12,358805065,place_of_worship,3.000000,418513730,restaurant,1.000000,0.750000
21,3986912498,fast_food,3.000000,3986855420,cafe,1.000000,0.750000
22,3986912498,fast_food,3.000000,411290725,post_box,1.000000,0.750000
33,2718062345,pharmacy,3.000000,4173572889,arts_centre,1.000000,0.750000
32,4018989416,bicycle_parking,2.019438,1630840817,restaurant,1.000000,0.668812
31,4018989416,bicycle_parking,2.019438,2718062345,pharmacy,3.000000,0.402323
